# Portfolio Management

A portfolio is a collection of financial investments. Let's see how we construct, analyse and optimise portfolios with Python. Like with every notebook, this one starts with a list of imports.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import yfinance as yf

plt.style.use("ggplot")

## Portfolio Formation

Today we'll form what is known as a 1/N portfolio, with 5 stocks in it. The 1/N refers to the total investment being divided equally among the N stocks that comprise the portfolio - which is 5 in our case today.

Depending on what kind of data we have, portfolio formation can involve different steps.

### Exercise: Pull a Portfolio (Yahoo! Finance)

In this exercise, you'll pull Yahoo Finance data into panel data using yfinance.

Here is an example of how we could join together data from multiple CSV files. Can you adapt this code to instead join together data pulled directly from Yahoo Finance?

```python
tickers = ["TSLA", "MSFT", "AMZN", "META", "NVDA"]
groups = []

for ticker in tickers:
    data = pd.read_csv(ticker + "_2020.csv")
    df.columns = df.columns.droplevel(1)
    data.insert(0, "Ticker", ticker)
    groups.append(data)

multi_df = pd.concat(data)
multi_df
```

Try it out in the code cell below:

In [2]:
## YOUR CODE GOES HERE

#### Multi-Stock Time Series Data (CRSP)

Maybe instead of Yahoo! Finance, you took advantage of the [College WRDS access](https://www.imperial.ac.uk/admin-services/library/subject-support/business/crsp/), or obtained a CSV with your data in some other way.

Let's work with our panel data as before.

In [3]:
df = pd.read_csv("../data/top_six_2020_2025.csv", dayfirst=True)

In [8]:
# Pivot the DataFrame to wide format with stocks as columns and dates as index
df_pivot = df.pivot(index='DlyCalDt', columns='Ticker', values='DlyClose')
df_pivot

portfolio_stocks = df_pivot.columns[:5] # Just grab the first 5
df_pivot = df_pivot[portfolio_stocks]

weights = np.ones(5) / 5

df_pivot

Ticker,AAPL,AMZN,GOOGL,META,MSFT
DlyCalDt,,,,,
01/02/2021,130.853821,167.143997,94.089218,260.780945,230.762817
01/02/2022,171.382431,151.193497,136.823410,317.503632,299.802063
01/02/2023,143.570236,105.150002,99.831276,152.401733,247.716675
01/02/2024,185.501541,159.279999,140.318466,392.928131,399.248016
01/03/2021,124.845673,157.307007,102.866081,263.667358,228.678787
...,...,...,...,...,...
31/10/2022,151.128387,102.440002,93.946579,92.723000,226.867950
31/10/2023,169.305481,133.089996,123.340294,299.856750,333.637909
31/10/2024,225.118683,186.399994,170.511078,566.618469,403.985077


## Calculating Portfolio Daily Returns

First, let's determine the returns of our portfolio. We'll need to start with the daily returns of each stock, and then use `np.dot()` to mutliply each column of returns against its weight in the portfolio.

In [5]:
# Calculate the simple return of each stock in the portfolio
returns = df_pivot.pct_change()
returns = returns.dropna()

# Compute the portfolio returns
portfolio_returns = np.dot(returns, weights)

# While we're at it check the daily volatility (standard deviation of portfolio returns)
daily_volatility = portfolio_returns.std()

## Annualising Returns and Volatility

To provide a broader picture of our portfolio's performance, let's calculate the annualised portfolio return and volatility. These are important metrics for evaluating and discussing portfolios. They are also used frequently in more advanced financial analyses.

In [6]:
# Define the number of trading days in a year
trading_days = 252

# Annualize the portfolio return
annualised_return = portfolio_returns.mean() * trading_days

# Annualize the portfolio volatility
annualised_volatility = daily_volatility * np.sqrt(trading_days)

## Sharpe Ratio

The Sharpe Ratio is a measure that helps investors understand the risk-adjusted return of an investment. A high Sharpe ratio indicates that the portfolio's returns are higher for each unit of risk taken on. In contrast, a lower Sharpe Ratio indicates a less favorable risk-reward trade-off, with the potential for lower returns relative to the amount of risk being assumed.

The Sharpe Ratio is calculated using annualised portfolio returns, portfolio volatility (as the measure of risk), and the *risk-free rate*. The risk-free rate is often derived from the yield of a theoretically risk-free investment, typically a government bond. The Sharpe Ratio is the average return earned in excess of the risk-free rate per unit of volatility or total risk.

### Exercise: Looking Sharpe

Calculate the Sharpe Ratio for our portfolio by applying the formula below.

$$ \text{Sharpe Ratio} = \frac{R_p - R_f}{\sigma_p} $$

- $R_p$ is our annualised portfolio return
- $R_f$ is the risk-free rate
- $\sigma_p$ is our annualised volatility


In [7]:
## YOUR CODE GOES HERE